In [6]:
import logging
import pandas as pd
import numpy as np
from numpy import random
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re



In [32]:
df = pd.read_csv('./data/yahoodata/ydata-ynacc-v1_0_expert_annotations.tsv',sep='\t')
df.head()

df.dropna(inplace=True)

,sdid,commentindex,headline,url,guid,commentid,timestamp,thumbs-up,thumbs-down,text,parentid,constructiveclass,sd_agreement,sd_type,sentiment,tone,commentagreement,topic,intendedaudience,persuasiveness
0,53971,2,Disneyland Worker Found Dead in Haunted Mansion,http://www.cosmopolitan.com/lifestyle/news/a56...,rjrPtwH5oVVuQnEXX3hf,00003n000000000000000000000000-ed2ae6d0-32ac-4...,1459917444,NaN,NaN,"These things happen , Every job has its dangers.",1459879464596-a3771c05-fd2e-4f44-a26a-23baec3b...,Constructive,NaN,Positive/respectful,negative,NaN,Disagreement with commenter,Off-topic with article,Reply to a specific commenter,Not persuasive
1,53971,0,Disneyland Worker Found Dead in Haunted Mansion,http://www.cosmopolitan.com/lifestyle/news/a56...,VaW6HEsuOFUAIBqjw1k~,1459879464596-a3771c05-fd2e-4f44-a26a-23baec3b...,1459879464,1.0,NaN,Sad to hear such a bad thing. Very dangerous j...,NaN,Constructive,NaN,Positive/respectful,mixed,NaN,NaN,Off-topic with article,Broadcast message / general audience,Not persuasive
2,53971,1,Disneyland Worker Found Dead in Haunted Mansion,http://www.cosmopolitan.com/lifestyle/news/a56...,uwQePj970KaMZuW3~9Q9,00002n000000000000000000000000-1c30b878-b717-4...,1459881644,NaN,NaN,Yes..because too many houses in EU look like t...,1459879464596-a3771c05-fd2e-4f44-a26a-23baec3b...,Constructive,NaN,Positive/respectful,neutral,Informative,NaN,Off-topic with article,Reply to a specific commenter,Not persuasive
3,135929,0,This Old Navy Ad Featuring an Interracial Fami...,http://mic.com/articles/142323/this-old-navy-a...,fixyWJivQjEQtPLLVXsu,1462203719963-3eeffb02-faae-4b51-9174-704c57e6...,1462203719,18.0,3.0,"I am frankly quite SICK of the phrase ""shoved ...",NaN,Not constructive,Agreement throughout,Off-topic/digression,negative,Mean,NaN,Off-topic with article,Broadcast message / general audience,Persuasive
4,135929,1,This Old Navy Ad Featuring an Interracial Fami...,http://mic.com/articles/142323/this-old-navy-a...,_TDnK715vO5y0OzZz_n4,00002I000000000000000000000000-7ef2ac58-bd84-4...,1462204643,7.0,2.0,"Ya, I always wonder why the conservatives are ...",1462203719963-3eeffb02-faae-4b51-9174-704c57e6...,Not constructive,Agreement throughout,Off-topic/digression,neutral,Sarcastic,Agreement with commenter,Off-topic with article,Reply to a specific commenter,Not persuasive


### Definition of ERICs
ERICs are characterized by argumentative, respectful exchanges containing persuasive, informative, and/or sympathetic comments. They tend to stay on topic with the original article and not to contain funny, mean, or sarcastic comments. We found differences between the distribution of annotations made by trained and untrained anno- tators, but high levels of agreement within each group, suggesting that crowdsourcing annotations for this task is reliable.

In [37]:
df = pd.read_csv('./data/yahoodata/ydata-ynacc-v1_0_IAC_annotations.tsv',sep='\t')
df.head()

,sdid,commentindex,headline,url,guid,commentid,timestamp,thumbs-up,thumbs-down,text,parentid,constructiveclass,sd_agreement,sd_type
0,7_convinceme,0,Montana vs. Texas,http://www.convinceme.net/debates/7/Montana-vs...,160,7_1,732680,0,0,Cockroaches die in -40.,7_1,Not constructive,Continual disagreement,Sarcastic|Off Topic
1,7_convinceme,1,Montana vs. Texas,http://www.convinceme.net/debates/7/Montana-vs...,50,7_2,732680,0,0,All I've got to say is -40 degrees. People wer...,7_1,Not constructive,Continual disagreement,Sarcastic|Off Topic
2,7_convinceme,2,Montana vs. Texas,http://www.convinceme.net/debates/7/Montana-vs...,28,7_3,732680,0,0,Frito Pie. Brilliant!,7_1,Not constructive,Continual disagreement,Sarcastic|Off Topic
3,7_convinceme,3,Montana vs. Texas,http://www.convinceme.net/debates/7/Montana-vs...,16,7_4,732708,0,0,I have been to Texas...and would rather not go...,7_1,Not constructive,Continual disagreement,Sarcastic|Off Topic
4,7_convinceme,4,Montana vs. Texas,http://www.convinceme.net/debates/7/Montana-vs...,312,7_5,733276,0,0,"everything's bigger in texas(hehe, penis joke.)",7_1,Not constructive,Continual disagreement,Sarcastic|Off Topic


In [21]:
df.head()

,sdid,headline,text,constructiveclass,sd_agreement,sd_type,sentiment,tone,commentagreement,topic,intendedaudience,persuasiveness
4,135929,This Old Navy Ad Featuring an Interracial Fami...,"Ya, I always wonder why the conservatives are ...",Not constructive,Agreement throughout,Off-topic/digression,neutral,Sarcastic,Agreement with commenter,Off-topic with article,Reply to a specific commenter,Not persuasive
78,70676,Trans Woman Attacked on NYC Train for Riding W...,A. Albarazi - Well that's your problem right t...,Not constructive,Continual disagreement,Argumentative (back and forth),negative,Mean,Disagreement with commenter,Off-topic with article,Reply to a specific commenter,Not persuasive
83,136165,This Old Navy Ad Featuring an Interracial Fami...,Ed Bolling = disgusting racist IDIOT. And you ...,Not constructive,Continual disagreement,Argumentative (back and forth),negative,"Controversial,Mean",Disagreement with commenter,"Off-topic with conversation,Off-topic with art...",Reply to a specific commenter,Not persuasive
85,136165,This Old Navy Ad Featuring an Interracial Fami...,"James - shut up moron, and go back to the zoo",Not constructive,Continual disagreement,Argumentative (back and forth),negative,Mean,Disagreement with commenter,"Off-topic with conversation,Off-topic with art...",Reply to a specific commenter,Not persuasive
89,136165,This Old Navy Ad Featuring an Interracial Fami...,Another coward. You created the welfare state ...,Not constructive,Continual disagreement,Argumentative (back and forth),negative,Sarcastic,Disagreement with commenter,Off-topic with article,Reply to a specific commenter,Not persuasive


In [29]:
X = df.text
y = df.sentiment
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)


3823     And the last excuse your #$%$ had.....because ...
23232    Not sure why you think the battery will only l...
12771    I think what @USERNAME is saying is that the m...
11947    I think these are general statements. My femal...
3342     No, not all "good" PR. I am sure they will los...
                               ...                        
19731    WHAT????  Oh Simpleton.  Never mind self expla...
21259    Newsflash idiot, 99.99% of the black race vote...
19057    People back then were lucky to live past 30. @...
6722     remember it is the criminal doing most gun vio...
12981    The Doobie Brothers? Why not Hootie and the Bl...
Name: text, Length: 1509, dtype: object

In [31]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

sentiments = ['mixed','neutral','positive','negative']

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(X_train, y_train)

from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

y_pred
# print('accuracy %s' % accuracy_score(y_pred, y_test))
# print(classification_report(y_test, y_pred,target_names=sentiments))

array(['negative', 'negative', 'negative', ..., 'negative', 'negative',
       'negative'], dtype='<U8')